### 📌 유저의 KnowledgeTag 별 정답률 계산

In [ ]:
%load_ext nb_black
%load_ext lab_black

import numpy as np
import pandas as pd
import os

from tqdm import tqdm

In [ ]:
# TODO : 원하는 데이터 경로 지정하기
DATA_PATH = "/opt/ml/input/data/FE_total_data.csv"
raw_data = pd.read_csv(DATA_PATH, parse_dates=["Timestamp"])
user_id = sorted(raw_data["userID"].unique())  # 유저 아이디 목록 리스트로 저장

In [ ]:
acc_by_kt_dict = dict()
"""
acc_by_kt_dict : 각 유저의 KT별 정답률을 저장한 dictionary
    - Key : 유저 아이딩
    - Value : 리스트 두개
        - 첫번째 리스트 : KT를 담고 있는 리스트
        - 두번째 리스트 : 첫번째 리스트에 동일한 인덱스에 있는 KT의 정답률
"""
for user in user_id:
    acc_by_kt_dict[user] = [[] for _ in range(2)]

for user in tqdm(user_id):  # 모든 유저에 대해
    data = raw_data[raw_data["userID"] == user]  # 각 유저에 대한 행만 추출하여 data에 저장
    kt_l = data["KnowledgeTag"].unique()  # 행들에 있는 고유한 KT 추출 ex) [7724, 7720, 7640]

    for i in kt_l:  # 유저가 풀어본 각 KT 대해
        kt = int(i)
        kt_correct = 0
        kt_total = 0
        for j in np.where(data["KnowledgeTag"] == i)[0]:  # 해당 KT인 문제만 순회하면서
            # for j in range(len(data)) :
            if data["KnowledgeTag"].iloc[j] == i:  # 동일한 KT에 대해
                if (
                    data["answerCode"].iloc[j] == 1
                ):  # 정답을 맞췄으면, correct, total count + 1
                    kt_correct += 1
                    kt_total += 1
                else:
                    kt_total += 1  # 맞추지 못했으면 total count만 + 1
        acc_by_kt_dict[user][0].append(kt)  # 해당 KT는 KT 리스트에 저장
        acc_by_kt_dict[user][1].append(kt_correct / kt_total)  # KT에 대한 정답률 저장

In [ ]:
"""
acc_by_kt_category_list 
: 해당 리스트는 feature를 추가하기 전에 feature 값들을 일렬로 저장하기 위해 만든 리스트입니다.
"""
acc_by_kt_category_list = list()

for i in tqdm(range(len(raw_data))):  # 모든 행(instance)에 대해
    u = raw_data["userID"].iloc[i]  # 유저를 확인하고
    kt = raw_data["KnowledgeTag"].iloc[i]  # 대분류를 확인하고
    idx = np.where(acc_by_kt_dict[u][0] == kt)[0][
        0
    ]  # 해당 유저의 해당 대분류에 대한 정답률을 위에서 정의한 dictionary에서 찾아서
    acc = acc_by_kt_dict[u][1][idx]
    acc_by_kt_category_list.append(acc)  # 리스트에 저장한다.

raw_data["KnowledgeTagAcc"] = acc_by_kt_category_list  # 리스트에 KnowledgeTagAcc feature 추가

In [ ]:
"""
acc_cate_list : 각 대분류 정답률을 카테고리로 만들어 저장하려고 한 리스트. 
ex. 원래 값 [0.04, 0.79, 0.43, 0.99] -> 곱하기 10 : [0.4, 7.9, 4.3, 9.9] -> round : [0, 8, 4, 10] -> 더하기 1 : [1, 9, 5, 11]
"""
acc_cate_list = list()
for tag_acc in tqdm(raw_data["KnowledgeTagAcc"]):
    acc = tag_acc * 10
    acc = round(acc) + 1
    acc_cate_list.append(acc)

raw_data["KTAccuracyCate"] = acc_cate_list  # 리스트로 저장

In [ ]:
raw_data.head()

In [ ]:
# TODO : 이름과 경로 원하는 데로 변경하기
os.makedirs("/opt/ml/input/data/FE", exist_ok=True)

raw_data.to_csv(
    "/opt/ml/input/data/FE/KnowledgeTagAcc.csv",
    columns=[
        "userID",
        "assessmentItemID",
        "testId",
        "answerCode",
        "Timestamp",
        "KnowledgeTag",
        "dataset",
        "KnowledgeTagAcc",
    ],
    index=False,
)  # 최종 결과 csv로 반환

raw_data.to_csv(
    "/opt/ml/input/data/FE/KTAccuracyCate.csv",
    columns=[
        "userID",
        "assessmentItemID",
        "testId",
        "answerCode",
        "Timestamp",
        "KnowledgeTag",
        "dataset",
        "KTAccuracyCate",
    ],
    index=False,
)  # 최종 결과 csv로 반환